In [1]:
!pip install plotly prophet --quiet


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:

# 1) monitoring Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2) import
import pandas as pd
import numpy as np

# 3) read datasets from drive
base = '/content/drive/MyDrive/m5/data/raw/'
sales    = pd.read_csv(base + 'sales_train_validation.csv')
calendar = pd.read_csv(base + 'calendar.csv')
prices   = pd.read_csv(base + 'sell_prices.csv')

# 4) Loot
day_cols = [c for c in sales.columns if c.startswith('d_')]

# 5) check
print("sales:",    sales.shape)
print("calendar:", calendar.shape)
print("prices:",   prices.shape)
print("day_cols count:", len(day_cols))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
sales: (30490, 1919)
calendar: (1969, 14)
prices: (6841121, 4)
day_cols count: 1913


In [5]:
# 1) Let's look at the column types in sales
print("Types of columns in sales:")
print(sales.dtypes.value_counts(), "\n")

# 2) Check for omissions
# 3) Unique values for key fields
total_missing = sales.isna().sum().sum()
print(f"Total missing values in sales: {total_missing}\n")

missing_per_col = sales.isna().sum()
print("Тоp-5 columns by number of skips:")
print(missing_per_col.sort_values(ascending=False).head(), "\n")

# 3) Unique values for key fields
print("Unique item_id:",    sales['item_id'].nunique())
print("Unique dept_id:",    sales['dept_id'].nunique())
print("Unique store_id:",   sales['store_id'].nunique())
print("Unique state_id:",   sales['state_id'].nunique(), "\n")

# 4) Let us briefly familiarize ourselves with “days”
# d_1...d_1913 are days from the beginning of the sample.
day_cols = [c for c in sales.columns if c.startswith('d_')]
print(f"Total “daytime” columns: {len(day_cols)}")
print("The first 5 names of such columns:Total “day” columns:", day_cols[:5])
print("Last 5 titles:", day_cols[-5:])


Types of columns in sales:
int64     1913
object       6
Name: count, dtype: int64 

Total missing values in sales: 0

Тоp-5 columns by number of skips:
d_1913     0
id         0
item_id    0
dept_id    0
cat_id     0
dtype: int64 

Unique item_id: 3049
Unique dept_id: 7
Unique store_id: 10
Unique state_id: 3 

Total “daytime” columns: 1913
The first 5 names of such columns:Total “day” columns: ['d_1', 'd_2', 'd_3', 'd_4', 'd_5']
Last 5 titles: ['d_1909', 'd_1910', 'd_1911', 'd_1912', 'd_1913']


In [6]:
# 1) Find all columns that start with "d_"
day_cols = [c for c in sales.columns if c.startswith('d_')]
print(f"Total day columns: {len(day_cols)} (from {day_cols[0]} to {day_cols[-1]})")

# 2) Take a sample: first 5 series and first 30 days
sample = sales.loc[:4, ['id'] + day_cols[:30]]

# 3) Melt the DataFrame from wide to long format
melted = sample.melt(
    id_vars='id',       # keep the 'id' column as is
    var_name='d',       # name of the column containing day identifiers
    value_name='sales'  # name of the column containing sales values
)
print("After melt shape:", melted.shape)
display(melted.head(8))

# 4) Merge with calendar to attach real dates, weekday, and month
merged = (
    melted
    .merge(
        calendar[['d', 'date', 'wday', 'month']],
        on='d',
        how='left'
    )
)
print("After merge shape:", merged.shape)
display(merged.head(8))



Total day columns: 1913 (from d_1 to d_1913)
After melt shape: (150, 3)


,id,d,sales
0,HOBBIES_1_001_CA_1_validation,d_1,0
1,HOBBIES_1_002_CA_1_validation,d_1,0
2,HOBBIES_1_003_CA_1_validation,d_1,0
3,HOBBIES_1_004_CA_1_validation,d_1,0
4,HOBBIES_1_005_CA_1_validation,d_1,0
5,HOBBIES_1_001_CA_1_validation,d_2,0
6,HOBBIES_1_002_CA_1_validation,d_2,0
7,HOBBIES_1_003_CA_1_validation,d_2,0


After merge shape: (150, 6)


,id,d,sales,date,wday,month
0,HOBBIES_1_001_CA_1_validation,d_1,0,2011-01-29,1,1
1,HOBBIES_1_002_CA_1_validation,d_1,0,2011-01-29,1,1
2,HOBBIES_1_003_CA_1_validation,d_1,0,2011-01-29,1,1
3,HOBBIES_1_004_CA_1_validation,d_1,0,2011-01-29,1,1
4,HOBBIES_1_005_CA_1_validation,d_1,0,2011-01-29,1,1
5,HOBBIES_1_001_CA_1_validation,d_2,0,2011-01-30,2,1
6,HOBBIES_1_002_CA_1_validation,d_2,0,2011-01-30,2,1
7,HOBBIES_1_003_CA_1_validation,d_2,0,2011-01-30,2,1


In [7]:
# 1) List files in current directory to confirm sales_train_validation.csv is present
!ls -lh

# 2) If not present, mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 3) Read all three CSVs from Drive
import pandas as pd
base = '/content/drive/MyDrive/m5/data/raw/'

sales    = pd.read_csv(base + 'sales_train_validation.csv')
calendar = pd.read_csv(base + 'calendar.csv')
prices   = pd.read_csv(base + 'sell_prices.csv')

print("sales shape:",    sales.shape)
print("calendar shape:", calendar.shape)
print("prices shape:",   prices.shape)

# 4) Melt from wide → long
day_cols = [c for c in sales.columns if c.startswith('d_')]
melted = sales.melt(id_vars='id', var_name='d', value_name='sales')
print("After melt:", melted.shape)

# 5) Merge with calendar and parse date
merged = melted.merge(calendar[['d','date','wday','month']], on='d', how='left')
merged['date'] = pd.to_datetime(merged['date'])
print("After merge:", merged.shape)

# 6) Aggregate average sales by month and by weekday
avg_month = (
    merged
    .groupby(merged['date'].dt.month)['sales']
    .mean()
    .reset_index(name='avg_sales')
)
avg_wday = (
    merged
    .groupby('wday')['sales']
    .mean()
    .reset_index(name='avg_sales')
)

# 7) Plot with Plotly
import plotly.express as px

fig1 = px.bar(
    avg_month,
    x='date',
    y='avg_sales',
    labels={'date':'Month','avg_sales':'Average Sales'},
    title='Average Sales by Month'
)
fig1.show()

fig2 = px.line(
    avg_wday,
    x='wday',
    y='avg_sales',
    labels={'wday':'Weekday','avg_sales':'Average Sales'},
    title='Average Sales by Weekday'
)
fig2.show()


total 8.0K
drwx------ 5 root root 4.0K Jun 21 10:38 drive
drwxr-xr-x 1 root root 4.0K Jun 18 13:35 sample_data
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
sales shape: (30490, 1919)
calendar shape: (1969, 14)
prices shape: (6841121, 4)
After melt: (58479820, 3)
After merge: (58479820, 6)


In [8]:
# 1) Read sample submission
sample_sub = pd.read_csv(base + 'sample_submission.csv')

# 2) Create a mapping from series ID (with '_validation') to last-day sales
last_day_map = dict(zip(sales['id'], sales['d_1913']))

# 3) Normalize IDs: change any '_evaluation' → '_validation'
base_ids = sample_sub['id'].str.replace('_evaluation', '_validation')
sample_sub['last_sales'] = base_ids.map(last_day_map)

# 4) Fill F1–F28 with the mapped last-day sales
for col in sample_sub.columns[1:-1]:   # skip 'id' and 'last_sales'
    sample_sub[col] = sample_sub['last_sales']

# 5) Save naive submission
sample_sub.drop(columns='last_sales', inplace=True)
sample_sub.to_csv('submission_naive.csv', index=False)
print("Naive submission saved to submission_naive.csv")

# 6) Compute RMSE on last 28 days of training
import numpy as np

holdout     = sales[day_cols[-28:]].values       # shape (N, 28)
forecast    = sales['d_1913'].values             # length N
y_true      = holdout.flatten()                  # length N*28
y_pred      = np.repeat(forecast, 28)            # same length

rmse = np.sqrt(((y_true - y_pred) ** 2).mean())
print(f"Naive RMSE on last 28 days: {rmse:.4f}")


Naive submission saved to submission_naive.csv
Naive RMSE on last 28 days: 2.8092


In [9]:
# ————————————————————————————————
# 1) Compute weights as share of total sales per series
total_sales = sales[day_cols].sum(axis=1)        # shape (N,)
w = total_sales / total_sales.sum()             # normalized weights, shape (N,)

# Quick check:  ≈ 1.0
print("Sum of weights:", w.sum())


Sum of weights: 1.0


In [10]:
import numpy as np

# 2) Compute scale per series (unchanged)
train_vals = sales[day_cols].values
diffs      = np.diff(train_vals, axis=1)
scale      = np.mean(diffs**2, axis=1)

# 3) MSE on last 28 days
holdout    = sales[day_cols[-28:]].values
forecast   = sales['d_1913'].values
mse_series = np.mean((holdout - forecast[:, None])**2, axis=1)

# 4) RMSSE and WRMSSE
rmsse      = np.sqrt(mse_series / scale)
wrmsse     = np.sum(w * rmsse)  # since w.sum() = 1
print(f"Baseline Naive WRMSSE (sales-share weights): {wrmsse:.4f}")


Baseline Naive WRMSSE (sales-share weights): 1.0014


In [11]:
# Feature Engineering on a 100‐series sample for speed

# 2.1) Take a subset of 100 series
sample_ids = sales['id'].unique()[:100]

# 2.2) Melt to long form
df = (
    sales[sales['id'].isin(sample_ids)]
    .melt(
        id_vars=['id'],
        value_vars=day_cols,
        var_name='d',
        value_name='sales'
    )
)

# 2.3) Merge calendar features
df = df.merge(
    calendar[['d','date','wday','month','year','event_name_1','event_name_2']],
    on='d',
    how='left'
)

# 2.4) Sort and create lag / rolling features
df = df.sort_values(['id','date'])
df['lag_1']       = df.groupby('id')['sales'].shift(1)
df['lag_7']       = df.groupby('id')['sales'].shift(7)
df['roll_mean_7'] = (
    df.groupby('id')['lag_1']
      .rolling(7)
      .mean()
      .reset_index(level=0, drop=True)
)
df['roll_mean_28'] = (
    df.groupby('id')['lag_1']
      .rolling(28)
      .mean()
      .reset_index(level=0, drop=True)
)

# 2.5) Inspect the engineered features
display(df.head(10))


,id,d,sales,date,wday,month,year,event_name_1,event_name_2,lag_1,lag_7,roll_mean_7,roll_mean_28
0,HOBBIES_1_001_CA_1_validation,d_1,0,2011-01-29,1,1,2011,NaN,NaN,NaN,NaN,NaN,NaN
100,HOBBIES_1_001_CA_1_validation,d_2,0,2011-01-30,2,1,2011,NaN,NaN,0.0,NaN,NaN,NaN
200,HOBBIES_1_001_CA_1_validation,d_3,0,2011-01-31,3,1,2011,NaN,NaN,0.0,NaN,NaN,NaN
300,HOBBIES_1_001_CA_1_validation,d_4,0,2011-02-01,4,2,2011,NaN,NaN,0.0,NaN,NaN,NaN
400,HOBBIES_1_001_CA_1_validation,d_5,0,2011-02-02,5,2,2011,NaN,NaN,0.0,NaN,NaN,NaN
500,HOBBIES_1_001_CA_1_validation,d_6,0,2011-02-03,6,2,2011,NaN,NaN,0.0,NaN,NaN,NaN
600,HOBBIES_1_001_CA_1_validation,d_7,0,2011-02-04,7,2,2011,NaN,NaN,0.0,NaN,NaN,NaN
700,HOBBIES_1_001_CA_1_validation,d_8,0,2011-02-05,1,2,2011,NaN,NaN,0.0,0.0,0.0,NaN
800,HOBBIES_1_001_CA_1_validation,d_9,0,2011-02-06,2,2,2011,SuperBowl,NaN,0.0,0.0,0.0,NaN
900,HOBBIES_1_001_CA_1_validation,d_10,0,2011-02-07,3,2,2011,NaN,NaN,0.0,0.0,0.0,NaN


In [2]:
print('sales in globals ->', 'sales' in globals())
print('day_cols length  ->', len(day_cols) if 'day_cols' in globals() else None)


sales in globals -> False
day_cols length  -> None


In [3]:

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd, numpy as np

base = '/content/drive/MyDrive/m5/data/raw/'
sales    = pd.read_csv(base + 'sales_train_validation.csv')
calendar = pd.read_csv(base + 'calendar.csv')
prices   = pd.read_csv(base + 'sell_prices.csv')

day_cols = [c for c in sales.columns if c.startswith('d_')]

print("sales:", sales.shape,
      "\ncalendar:", calendar.shape,
      "\nprices:", prices.shape,
      "\nday_cols:", len(day_cols))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
sales: (30490, 1919) 
calendar: (1969, 14) 
prices: (6841121, 4) 
day_cols: 1913


In [2]:
# ——— master-init
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd, numpy as np


base = '/content/drive/MyDrive/m5/data/raw/'
sales    = pd.read_csv(base + 'sales_train_validation.csv')
calendar = pd.read_csv(base + 'calendar.csv')


day_cols = [c for c in sales.columns if c.startswith('d_')]


N_SERIES   = 2000
subset_ids = sales['id'].unique()[:N_SERIES]
sales_sub  = sales[sales['id'].isin(subset_ids)].copy()
sales_sub[day_cols] = sales_sub[day_cols].astype('int16')

print("sales_sub:", sales_sub.shape,
      "\ncalendar:", calendar.shape,
      "\nday_cols:", len(day_cols))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
sales_sub: (2000, 1919) 
calendar: (1969, 14) 
day_cols: 1913


In [5]:
# 3-A-1) Build long DataFrame with calendar features
df = (
    sales_sub
      .melt(id_vars=['id'],
            value_vars=day_cols,
            var_name='d',
            value_name='sales')
      .merge(calendar[['d','date','wday','month','year']],
             on='d', how='left')
      .sort_values(['id','date'])
      .reset_index(drop=True)
)

# 3-A-2) Lag & rolling features
df['lag_1']        = df.groupby('id')['sales'].shift(1)
df['lag_7']        = df.groupby('id')['sales'].shift(7)
df['roll_mean_7']  = (
    df.groupby('id')['lag_1']
      .rolling(7).mean()
      .reset_index(level=0, drop=True)
)
df['roll_mean_28'] = (
    df.groupby('id')['lag_1']
      .rolling(28).mean()
      .reset_index(level=0, drop=True)
)

# 3-A-3) Drop rows with NaNs in feature columns
feat_cols = ['lag_1','lag_7','roll_mean_7','roll_mean_28',
             'wday','month','year']
df = df.dropna(subset=feat_cols)
df['date'] = pd.to_datetime(df['date'])


print("Prepared DF shape:", df.shape)


Prepared DF shape: (3770000, 11)


In [7]:
!pip install -U lightgbm --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 48.6 MB/s eta 0:00:00


In [10]:
import lightgbm as lgb, numpy as np, pandas as pd
from datetime import timedelta

# ——— split ———
last_date = df['date'].max()
val_start = last_date - timedelta(days=27)
train_df  = df[df['date'] <  val_start]
valid_df  = df[df['date'] >= val_start]

feat_cols = ['lag_1','lag_7','roll_mean_7','roll_mean_28','wday','month','year']
X_train, y_train = train_df[feat_cols], train_df['sales']
X_val,   y_val   = valid_df[feat_cols], valid_df['sales']

dtrain = lgb.Dataset(X_train, label=y_train)
dval   = lgb.Dataset(X_val,   label=y_val)

params = dict(
    objective='regression',
    metric='rmse',
    learning_rate=0.1,
    num_leaves=31,
    feature_fraction=0.8,
    verbose=-1
)

model = lgb.train(
    params,
    dtrain,
    num_boost_round=500,
    valid_sets=[dval],
    callbacks=[lgb.early_stopping(stopping_rounds=50)]
)

# RMSE
y_pred = model.predict(X_val, num_iteration=model.best_iteration)
rmse_val = np.sqrt(((y_val - y_pred) ** 2).mean())
print(f"Validation RMSE: {rmse_val:.4f}")


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[97]	valid_0's rmse: 1.69237
Validation RMSE: 1.6924


In [11]:

total_sales_sub = sales_sub[day_cols].sum(axis=1)
w_sub = total_sales_sub / total_sales_sub.sum()

train_vals_sub = sales_sub[day_cols].values.astype(np.float32)
scale_sub = np.mean(np.diff(train_vals_sub, axis=1)**2, axis=1)

weight_map = dict(zip(sales_sub['id'], w_sub))
scale_map  = dict(zip(sales_sub['id'], scale_sub))

valid_df = valid_df.copy()
valid_df['pred']  = y_pred
valid_df['w']     = valid_df['id'].map(weight_map)
valid_df['scale'] = valid_df['id'].map(scale_map)

import numpy as np, pandas as pd
rmsse_series = (
    valid_df
      .groupby('id')
      .apply(lambda g: np.sqrt(((g.sales - g.pred)**2).mean() /
                               g.scale.iloc[0]))
)

wrmsse_val = (rmsse_series * pd.Series(weight_map)).sum()
print(f"Validation WRMSSE (subset): {wrmsse_val:.4f}")


Validation WRMSSE (subset): 0.7428


/tmp/ipython-input-11-2350790796.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: np.sqrt(((g.sales - g.pred)**2).mean() /
